In [ ]:
def prediction_func(currency_name):
    from textblob import TextBlob
    from bs4 import BeautifulSoup
    import requests

    class Analysis:
        def _init_(self, term):
            self.term = term
            self.sentiment = 0
            self.subjectivity = 0
            self.url = 'https://www.google.com/search?q={0}&source=lnms&tbm=nws'.format(self.term)

        def run(self):
            response = requests.get(self.url)
            soup = BeautifulSoup(response.text, 'html.parser')
            headline_results = soup.find_all('div', class_='kCrYT')
            for text in headline_results:
                blob = TextBlob(text.get_text())    
                self.sentiment += blob.sentiment.polarity / len(headline_results)
                self.subjectivity += blob.sentiment.subjectivity / len(headline_results)
            return (self.sentiment, self.subjectivity)
            
    #a= Analysis('bitcoin')
    #a.run()
    #print(a.term, a.subjectivity, a.sentiment)


    import yfinance as yf
    import os
    import pandas as pd

    btc_ticker = yf.Ticker(currency_name+"-USD")
    btc=btc_ticker.history(period="max")
    btc.to_csv("btc.csv")

    del btc['Dividends']
    del btc['Stock Splits']

    import requests
    from datetime import datetime
    import yfinance as yf
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error



    # Fetch real-time data using CoinGecko API
    if currency_name=="BTC":
      crypto_id='bitcoin'
    if currency_name=="ETH":
      crypto_id='ethereum'
    if currency_name=="XRP":
      crypto_id='ripple'
    if currency_name=="LTC":
      crypto_id='litecoin'
    if currency_name=="USDC":
      crypto_id='usdcoin'
    if currency_name=="USDT":
      crypto_id='usdt'

    #crypto_id = currency_name  # Example: 'bitcoin'
    vs_currency = 'usd'  # Example: 'usd'
    real_time_url = f'https://api.coingecko.com/api/v3/simple/price?ids={crypto_id}&vs_currencies={vs_currency}'
    real_time_response = requests.get(real_time_url)
    real_time_data = real_time_response.json()
    current_price = real_time_data[crypto_id][vs_currency]


    # Define a function to calculate the sentiment score for a given date
    def get_sentiment(date):
        term = currency_name+'price ' + date.strftime('%Y-%m-%d')
        analysis = Analysis(term)
        sentiment, subjectivity = analysis.run()
        return sentiment

    # Add a new column to the historical data for the sentiment score
    btc['Sentiment'] = 0

    # Calculate the sentiment score for each day in the historical data
    for date in btc.index[-8:-1:-1]:
        sentiment = get_sentiment(date)
        btc.at[date, 'Sentiment'] = sentiment

    # Prepare the historical data for model training
    # Extract features and target variable
    X = btc.drop(['Close'], axis=1)  # Features (excluding Close price)
    y = btc['Close']  # Target variable (Close price)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train an ensemble model (e.g., RandomForestRegressor) using historical data
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Use the trained model to make predictions
    # Prepare current data for prediction
    current_data = pd.DataFrame(columns=X.columns)  # Create an empty DataFrame with the same columns as historical data
    current_data = current_data.append(pd.Series(current_price, index=current_data.columns), ignore_index=True)  # Append current price as a row to the DataFrame

    # Make predictions using the trained model
    prediction = model.predict(current_data)

    # Print the predicted Bitcoin price
    #print(f'Predicted price: {prediction[0]}')
    return (prediction[0])

print("BITCOIN PREDICTED PRICE: ")
print(prediction_func("BTC"))

print("\nETHEREUM PREDICTED PRICE: ")
print(prediction_func("ETH"))

print("\nLITECOIN  PREDICTED PRICE: ")
print(prediction_func("LTC"))

print("\nRIPPLE  PREDICTED PRICE: ")
print(prediction_func("XRP"))




BITCOIN PREDICTED PRICE: 


<ipython-input-5-0a7be6f00283>:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_data = current_data.append(pd.Series(current_price, index=current_data.columns), ignore_index=True)  # Append current price as a row to the DataFrame


27983.133828125

ETHEREUM PREDICTED PRICE: 


<ipython-input-5-0a7be6f00283>:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_data = current_data.append(pd.Series(current_price, index=current_data.columns), ignore_index=True)  # Append current price as a row to the DataFrame


1859.6545629882812

LITECOIN  PREDICTED PRICE: 


<ipython-input-5-0a7be6f00283>:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_data = current_data.append(pd.Series(current_price, index=current_data.columns), ignore_index=True)  # Append current price as a row to the DataFrame


91.34170166015625

RIPPLE  PREDICTED PRICE: 
0.5103493738174438


<ipython-input-5-0a7be6f00283>:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_data = current_data.append(pd.Series(current_price, index=current_data.columns), ignore_index=True)  # Append current price as a row to the DataFrame
